In [12]:
import numpy as np
import pandas as pd

In [13]:
df = pd.read_excel('vals.xlsx')

In [14]:
df.sort_values('1/ratio',ascending=False)

,Element,Error %,Cation Radii,Anion Radii,k2(ra+rb)-1,exponent,1/exp,rate dcn/dr,partial diff,anion to cation,1/ratio
7,CuI,3.573920,112.0,133,325.666667,0.593887,0.627397,369.644813,NaN,0.842105,1.187500
9,ZnS,2.499976,118.0,103,293.666667,0.625086,0.615352,364.824723,NaN,1.145631,0.872881
13,LiSnS2,3.439916,136.5,103,318.333333,0.600896,0.624650,368.834149,NaN,1.325243,0.754579
6,LiCl,5.121300,133.0,99,308.333333,0.610588,0.620891,367.455540,NaN,1.343434,0.744361
11,SnS2,1.096295,140.0,103,323.000000,0.596426,0.626399,369.369050,NaN,1.359223,0.735714
10,CoO2,1.805294,111.0,63,231.000000,0.691011,0.591362,343.816495,NaN,1.761905,0.567568
12,LiCoO2,5.560246,122.0,63,245.666667,0.674984,0.597021,350.337187,NaN,1.936508,0.516393
8,LiF,4.715887,133.0,64,261.666667,0.657923,0.603164,356.262869,NaN,2.078125,0.481203
5,CsCl,3.230739,232.0,99,440.333333,0.494339,0.669192,365.385611,NaN,2.343434,0.426724
0,LiH,14.133308,133.0,32,219.000000,0.704406,0.586715,337.639621,NaN,4.156250,0.240602


In [15]:
#!/usr/bin/env python3
import os
import sys
from ase.io import read, write

def convert_vasp_to_xyz(vasp_file, xyz_file=None):
    """Convert VASP structure file to XYZ format."""
    if xyz_file is None:
        xyz_file = os.path.splitext(vasp_file)[0] + '.xyz'
    
    # Read the VASP structure
    atoms = read(vasp_file, format='vasp')
    
    # Write to XYZ format
    write(xyz_file, atoms, format='xyz')
    
    print(f"Converted {vasp_file} to {xyz_file}")
    return xyz_file

# If run as a script
if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python vasp_to_xyz.py file1.vasp [file2.vasp ...]")
        sys.exit(1)
    

    convert_vasp_to_xyz(vasp_file='LiTiS2.vasp')


Converted LiTiS2.vasp to LiTiS2.xyz


In [16]:
import os
import sys
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import display, HTML
from ase.io import read, write

# Set up matplotlib for better display in notebook
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

def display_status(message, is_error=False):
    """Display status messages with styling"""
    if is_error:
        display(HTML(f"<div style='color: red; font-weight: bold;'>{message}</div>"))
    else:
        display(HTML(f"<div style='color: green;'>{message}</div>"))
        
def check_dftd3_installation():
    """Check if dftd3 is properly installed and accessible"""
    try:
        result = subprocess.run(['s-dftd3', '--version'], 
                               capture_output=True, text=True)
        if result.returncode != 0:
            display_status("Warning: dftd3 command exists but may not be functioning properly", True)
            return False
        display_status(f"DFTD3 is installed: {result.stdout.strip() if result.stdout else 'Unknown version'}")
        return True
    except FileNotFoundError:
        display_status("Error: dftd3 command not found. Please install it using conda install -c conda-forge dftd3", True)
        return False


In [17]:
def convert_vasp_to_xyz(vasp_file, xyz_file=None):
    """Convert VASP structure file to XYZ format."""
    if xyz_file is None:
        # Use the same base name but with .xyz extension
        xyz_file = os.path.splitext(vasp_file)[0] + '.xyz'
    
    try:
        # Read the VASP structure
        atoms = read(vasp_file, format='vasp')
        
        # Write to XYZ format
        write(xyz_file, atoms, format='xyz')
        
        display_status(f"Converted {vasp_file} to {xyz_file}")
        return xyz_file
    except Exception as e:
        display_status(f"Error converting {vasp_file} to XYZ: {str(e)}", True)
        return None

def test_conversion():
    """Test function to check if ASE can handle VASP files correctly"""
    from tempfile import NamedTemporaryFile
    
    # Create a minimal VASP file for testing
    vasp_content = """Test system
1.0
        3.0000000000         0.0000000000         0.0000000000
        0.0000000000         3.0000000000         0.0000000000
        0.0000000000         0.0000000000         3.0000000000
   H
   1
Direct
     0.000000000         0.000000000         0.000000000
"""
    try:
        with NamedTemporaryFile(suffix='.vasp', delete=False) as f:
            f.write(vasp_content.encode('utf-8'))
            temp_file = f.name
        
        # Try to convert
        xyz_file = convert_vasp_to_xyz(temp_file)
        
        # Clean up
        os.remove(temp_file)
        if xyz_file and os.path.exists(xyz_file):
            os.remove(xyz_file)
            
        return True
    except Exception as e:
        display_status(f"ASE could not process VASP files: {str(e)}", True)
        return False


In [18]:
def extract_c6_parameters(dftd3_output):
    """Parse DFTD3 output and extract C6 parameters and coordination numbers."""
    c6_data = []
    cn_data = {}
    reading_c6 = False
    reading_cn = False
    dftd3_energy = None
    
    for line in dftd3_output.splitlines():
        # Extract the DFT-D3 energy
        if "Edisp /kcal,au,eV" in line:
            parts = line.split()
            if len(parts) >= 4:
                try:
                    dftd3_energy = float(parts[3])  # in eV
                except:
                    pass
                
        # Look for coordination numbers
        if "Coordination numbers" in line:
            reading_cn = True
            continue
            
        if reading_cn:
            if line.strip() == "":
                reading_cn = False
                continue
                
            parts = line.split()
            if len(parts) >= 3:
                try:
                    atom_idx = int(parts[0])
                    element = parts[1]
                    cn_value = float(parts[2])
                    cn_data[atom_idx] = {'element': element, 'CN': cn_value}
                except (ValueError, IndexError):
                    continue
        
        # Look for C6 coefficients
        if "C6 coefficients and radii" in line:
            reading_c6 = True
            continue
            
        if reading_c6:
            if line.strip() == "" or "Coordination number CN" in line:
                reading_c6 = False
                continue
                
            parts = line.split()
            if len(parts) >= 6:
                try:
                    atom1 = int(parts[0])
                    atom2 = int(parts[1])
                    atom1_elem = parts[2]
                    atom2_elem = parts[3]
                    c6_value = float(parts[4])
                    r0_value = float(parts[5])
                    
                    c6_data.append({
                        'atom1': atom1,
                        'atom2': atom2,
                        'elem1': atom1_elem,
                        'elem2': atom2_elem,
                        'C6': c6_value,
                        'R0': r0_value
                    })
                except (ValueError, IndexError):
                    continue
    
    return c6_data, cn_data, dftd3_energy


In [19]:
def process_vasp_file(vasp_file, functional='pbe', use_pbc=True, save_files=True):
    """Process a VASP file, convert to XYZ, run DFTD3, and extract C6 parameters."""
    # Convert to XYZ
    xyz_file = convert_vasp_to_xyz(vasp_file)
    if not xyz_file:
        return None, None, None
    
    # Run DFTD3
    c6_data, cn_data, dftd3_energy = run_dftd3_on_structure(xyz_file, functional, use_pbc)
    
    if c6_data is None:
        return None, None, None
    
    # Create output file name
    output_file = os.path.splitext(vasp_file)[0] + '_c6_parameters.txt'
    
    # Write results to file if requested
    if save_files:
        with open(output_file, 'w') as f:
            f.write(f"C6 Parameters for {vasp_file}\n")
            f.write(f"DFT Functional: {functional}\n")
            f.write(f"Periodic Boundary Conditions: {'Yes' if use_pbc else 'No'}\n")
            if dftd3_energy is not None:
                f.write(f"Total DFT-D3 dispersion energy: {dftd3_energy:.6f} eV\n")
            f.write("\n")
            
            # First write coordination numbers
            f.write("Coordination Numbers:\n")
            f.write("Atom   Element  CN\n")
            f.write("-" * 25 + "\n")
            
            for atom_idx, data in sorted(cn_data.items()):
                f.write(f"{atom_idx:5d} {data['element']:8s} {data['CN']:10.4f}\n")
            
            f.write("\nC6 Parameters:\n")
            f.write("Atom1 Atom2 Element1 Element2 C6_Value(J·nm^6·mol^-1) R0(Å)\n")
            f.write("-" * 60 + "\n")
            
            for entry in c6_data:
                f.write(f"{entry['atom1']:5d} {entry['atom2']:5d} {entry['elem1']:7s} {entry['elem2']:7s} "
                        f"{entry['C6']:20.6f} {entry['R0']:10.6f}\n")
        
        display_status(f"C6 parameters written to {output_file}")
    
    return c6_data, cn_data, dftd3_energy


In [20]:
def analyze_c6_parameters(c6_data, cn_data, base_filename=None, save_files=True):
    """Analyze C6 parameters and generate visualization."""
    if not c6_data:
        display_status("No C6 data to analyze", True)
        return
        
    # Group C6 values by element pairs
    element_pairs = defaultdict(list)
    
    for entry in c6_data:
        elem_pair = tuple(sorted([entry['elem1'], entry['elem2']]))
        element_pairs[elem_pair].append(entry['C6'])
    
    # Calculate statistics for each element pair
    stats = {}
    for elem_pair, c6_values in element_pairs.items():
        stats[elem_pair] = {
            'mean': np.mean(c6_values),
            'std': np.std(c6_values),
            'min': np.min(c6_values),
            'max': np.max(c6_values),
            'count': len(c6_values)
        }
    
    # Create a DataFrame for better display in the notebook
    try:
        import pandas as pd
        
        # Prepare data for DataFrame
        stats_data = []
        for elem_pair, stat in sorted(stats.items()):
            pair_str = f"{elem_pair[0]}-{elem_pair[1]}"
            stats_data.append({
                'Pair': pair_str,
                'Count': stat['count'],
                'Mean C6': stat['mean'],
                'Std Dev': stat['std'],
                'Min': stat['min'],
                'Max': stat['max']
            })
        
        # Create and display DataFrame
        df = pd.DataFrame(stats_data)
        display(HTML("<h3>C6 Parameter Statistics by Element Pair</h3>"))
        display(df)
        
        # Write statistics to file if requested
        if save_files and base_filename:
            stats_file = f"{base_filename}_c6_statistics.txt"
            with open(stats_file, 'w') as f:
                f.write("C6 Parameter Statistics by Element Pair\n")
                f.write("-" * 60 + "\n")
                f.write("Pair      Count    Mean C6     Std Dev     Min        Max\n")
                f.write("-" * 60 + "\n")
                
                for elem_pair, stat in sorted(stats.items()):
                    pair_str = f"{elem_pair[0]}-{elem_pair[1]}"
                    f.write(f"{pair_str:10s} {stat['count']:5d} {stat['mean']:12.4f} {stat['std']:10.4f} "
                            f"{stat['min']:10.4f} {stat['max']:10.4f}\n")
            
            display_status(f"C6 statistics written to {stats_file}")
    except ImportError:
        # If pandas is not available, print the statistics directly
        display(HTML("<h3>C6 Parameter Statistics by Element Pair</h3>"))
        print("{:<10s} {:>5s} {:>12s} {:>10s} {:>10s} {:>10s}".format(
            "Pair", "Count", "Mean C6", "Std Dev", "Min", "Max"))
        print("-" * 60)
        
        for elem_pair, stat in sorted(stats.items()):
            pair_str = f"{elem_pair[0]}-{elem_pair[1]}"
            print("{:<10s} {:5d} {:12.4f} {:10.4f} {:10.4f} {:10.4f}".format(
                pair_str, stat['count'], stat['mean'], stat['std'], stat['min'], stat['max']))
    
    # Create visualization
    display(HTML("<h3>Mean C6 Parameters by Element Pair</h3>"))
    
    # Bar chart for mean C6 values
    plt.figure(figsize=(12, 8))
    pairs = [f"{p[0]}-{p[1]}" for p in stats.keys()]
    means = [s['mean'] for s in stats.values()]
    stds = [s['std'] for s in stats.values()]
    
    plt.bar(pairs, means, yerr=stds, capsize=5)
    plt.ylabel('C6 Value (J·nm^6·mol^-1)')
    plt.xlabel('Element Pair')
    plt.title('Mean C6 Parameters by Element Pair')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Save the plot if requested
    if save_files and base_filename:
        plot_file = f"{base_filename}_c6_plot.png"
        plt.savefig(plot_file)
        display_status(f"C6 visualization saved to {plot_file}")
    
    # Check if we have same-element pairs to analyze relation between CN and C6
    for element in set([cn['element'] for cn in cn_data.values()]):
        same_elem_pair = (element, element)
        if same_elem_pair in element_pairs:
            # Collect C6 and CN data for this element
            element_c6_data = []
            for entry in c6_data:
                if entry['elem1'] == element and entry['elem2'] == element:
                    cn1 = cn_data[entry['atom1']]['CN']
                    cn2 = cn_data[entry['atom2']]['CN']
                    element_c6_data.append({
                        'C6': entry['C6'],
                        'CN1': cn1,
                        'CN2': cn2,
                        'AvgCN': (cn1 + cn2) / 2
                    })
            
            if element_c6_data:
                display(HTML(f"<h3>C6 vs. Coordination Number for {element}-{element} Pairs</h3>"))
                
                plt.figure(figsize=(10, 6))
                avg_cns = [d['AvgCN'] for d in element_c6_data]
                c6_values = [d['C6'] for d in element_c6_data]
                
                plt.scatter(avg_cns, c6_values, alpha=0.7)
                plt.xlabel('Average Coordination Number')
                plt.ylabel('C6 Value (J·nm^6·mol^-1)')
                plt.title(f'C6 vs. Coordination Number for {element}-{element} Pairs')
                
                # Try to fit a trend line
                try:
                    from scipy import stats as scipy_stats
                    slope, intercept, r_value, p_value, std_err = scipy_stats.linregress(avg_cns, c6_values)
                    plt.plot(avg_cns, [slope * x + intercept for x in avg_cns], 'r-', 
                             label=f'y = {slope:.2f}x + {intercept:.2f} (R² = {r_value**2:.2f})')
                    plt.legend()
                except ImportError:
                    pass
                
                plt.tight_layout()
                plt.show()
                
                # Save the plot if requested
                if save_files and base_filename:
                    cn_plot_file = f"{base_filename}_{element}_c6_vs_cn.png"
                    plt.savefig(cn_plot_file)
                    display_status(f"C6 vs. CN plot for {element} saved to {cn_plot_file}")


In [21]:
def process_vasp_structures(vasp_files, functional='pbe', use_pbc=True, save_files=True):
    """Process multiple VASP files and extract C6 parameters."""
    results = {}
    
    # Check if dftd3 is installed
    if not check_dftd3_installation():
        display_status("Cannot proceed without DFTD3 installed", True)
        return None
    
    # Process each file
    for vasp_file in vasp_files:
        if not os.path.exists(vasp_file):
            display_status(f"File not found: {vasp_file}", True)
            continue
        
        display(HTML(f"<h2>Processing {vasp_file}</h2>"))
        c6_data, cn_data, dftd3_energy = process_vasp_file(vasp_file, functional, use_pbc, save_files)
        
        if c6_data and cn_data:
            results[vasp_file] = {
                'c6_data': c6_data,
                'cn_data': cn_data,
                'dftd3_energy': dftd3_energy
            }
            
            # Analyze the results
            analyze_c6_parameters(c6_data, cn_data, 
                                 base_filename=os.path.splitext(vasp_file)[0] if save_files else None,
                                 save_files=save_files)
        else:
            display_status(f"Failed to process {vasp_file}", True)
    
    return results


In [22]:
# Test if the environment is properly set up
# check_dftd3_installation()
# test_conversion()

# List of VASP files to process
vasp_files = [
    "LiTiS2.vasp",  # Replace with your actual VASP file name
    # Add more files as needed
]

# Choose functional - must match what was used in your VASP calculation
functional = "pbe"  # Options: pbe, revpbe, b3lyp, pbe0, etc.

# Run the processing
results = process_vasp_structures(
    vasp_files=vasp_files,
    functional=functional,
    use_pbc=True,  # Set to True for periodic systems, False for molecules
    save_files=True  # Save output files to disk
)


NameError: name 'run_dftd3_on_structure' is not defined